In [5]:
%matplotlib inline
import IPython.core.display         
# setup output image format (Chrome works best)
IPython.core.display.set_matplotlib_formats("svg")
import matplotlib.pyplot as plt
import matplotlib
from numpy import *
from sklearn import *
from scipy import stats
random.seed(100)
import pandas as pd
import csv
import numpy as np
import os

In [6]:
features_name=[
    # player 1 = big blind bets, always action player
    'preflop',
    'flop',
    'turn',
    'river',
    'private_card_1_number', # 0~12
    'private_card_1_suit', # d:0, c:1, h:2, s:3
    'private_card_2_number',
    'private_card_2_suit',
    'public_card_1_number',
    'public_card_1_suit',
    'public_card_2_number',
    'public_card_2_suit',
    'public_card_3_number',
    'public_card_3_suit',
    'public_card_4_number',
    'public_card_4_suit',
    'public_card_5_number',
    'public_card_5_suit',   
    'pot_ratio',
    'action',       # action
]

In [7]:
suit_values = {
    'd':1,
    'c':2,
    'h':3,
    's':4,
}

num_values = {
    '2':1,
    '3':2,
    '4':3,
    '5':4,
    '6':5,
    '7':6,
    '8':7,
    '9':8,
    'T':9,
    'J':10,
    'Q':11,
    'K':12,
    'A':13,
}

action_map = {
    'f':0,
    'c':1,
    'r':2,
}

def cards_array(s):
    a = np.zeros((4,13))
    if (isinstance(s, list)):
        for _s in s:
            a[suit_values[_s[1]]][num_values[_s[0]]] = 1
    else:
        a[suit_values[s[1]]][num_values[s[0]]] = 1
        
    return a

def read_actions(actions):
    action_array = []
    bets = ''
    for ac in actions:
        if ac == 'c' or ac == 'f' or ac == 'r':
            if bets != '':
                action_array.append(int(bets))
                bets = ''
            action_array.append(ac)
        else:
            bets += ac
    return action_array

def read_approach_1(line):
    state, rounds, all_actions, all_cards, bets, names = line.split(':')
    
    # read actions
    actions = all_actions.split('/')
    n_action = len(actions)
    is_p1 = True
    rows = []
    
    for i in range(n_action):
        row = np.zeros(20)
        # preflop, flop, turn, river
        row[i] = 1        
        action_array = read_actions(actions[i])
#         print(action_array)
        
        if len(action_array) == 0:
            break
        row[19] = action_map[action_array[0]]
        p2_bet = 100
        p1_bet = 50
        
        
        
        # read cards
        cards = all_cards.split('/')
        private_cards = cards[0]        
        row[4] = num_values[private_cards[0]]
        row[5] = suit_values[private_cards[1]]
        row[6] = num_values[private_cards[2]]
        row[7] = suit_values[private_cards[3]]
        
        if i > 1:
            row[8] = num_values[cards[1][:2][0]]
            row[9] = suit_values[cards[1][:2][1]]
            row[10] = num_values[cards[1][2:4][0]]
            row[11] = suit_values[cards[1][2:4][1]]
            row[12] = num_values[cards[1][4:6][0]]
            row[13] = suit_values[cards[1][4:6][1]]
            row[18] = 0.5
        if i > 2:
            row[14] = num_values[cards[2][0]]
            row[15] = suit_values[cards[2][1]]
        if i > 3:
            row[16] = num_values[cards[3][0]]
            row[17] = suit_values[cards[3][1]]
                    
        row[18] = 0.5
        rows.append(row)
        
    return rows

def position_of_next_dot(s):
    index = 0
    i = s.find('.')
    index = i
    index += s[i+1:].find('.')
    return index + 1

csv_name = 'converted_1'
# write CSV file
f = open(csv_name + '.csv', 'w')
writer = csv.writer(f)
writer.writerow(features_name)
f.close()

limit = 1
count = 0
name = ''
i = 0
for filename in os.listdir('logs_2pn_2017'):
    i += 1
    if i > 1:
        break
    if (filename[:position_of_next_dot(filename)] != name):
        name = filename[:position_of_next_dot(filename)]
        count = 0
    count+=1
    if count > limit:
        continue
    if not (filename.startswith("ASHE") and filename.endswith(".log")): 
        continue
        
    rows = []
    f = open("logs_2pn_2017\\" + filename, "r")
    for line in f: 
        # igonre not STATE
        if not line.startswith('STATE'):
            continue

        rows += read_approach_1(line)

    cf = open(csv_name + '.csv', 'a')
    writer = csv.writer(cf)
    writer.writerows(rows)

    cf.close()
    f.close()
    print('{} Saved to {}.csv'.format(filename, csv_name))
        
#     # read actions
#     actions = all_actions.split('/')
#     n_action = len(actions)
#     is_p1 = True
    
#     for i in range(n_action):
#         action = read_actions(actions[i])
#         print(action)
            
#     # read cards
#     cards = all_cards.split('/')
#     p1_hards, p2_hards = cards[0].split('|')
#     p1_card_1_number = num_values[p1_hards[0]]
#     p1_card_1_suit = suit_values[p1_hards[1]]
#     p1_card_2_number  = num_values[p1_hards[2]]
#     p1_card_2_suit = suit_values[p1_hards[3]]
#     p2_card_1_number = num_values[p2_hards[0]]
#     p2_card_1_suit = suit_values[p2_hards[1]]
#     p2_card_2_number = num_values[p2_hards[2]]
#     p2_card_2_suit = suit_values[p2_hards[3]]
    
    
#     # 2d represent cards
#     cards_list = [p1_hards, p2_hards]
#     if len(cards) > 1:
#         cards_list.append(cards[1][:2])
#         cards_list.append(cards[1][2:4])
#         cards_list.append(cards[1][4:6])
#     if len(cards) > 2:
#         cards_list.append(cards[2])
#     if len(cards) > 3:
#         cards_list.append(cards[3])
        
#     b = cards_array(cards_list)
#     print(b)
    
#     row = [
#         int(rounds), 
#         p1_card_1_number, 
#         p1_card_1_suit, 
#         p1_card_2_number, 
#         p1_card_2_suit, 
#         p2_card_1_number, 
#         p2_card_1_suit, 
#         p2_card_2_number, 
#         p2_card_2_suit, 
#     ]
#     rows.append(row)


ASHE.ElDonatoro.1.0.log Saved to converted_1.csv


In [8]:
# csv_name = 'converted_1'
# # write CSV file
# f = open(csv_name + '.csv', 'w')
# writer = csv.writer(f)
# writer.writerow(features_name)
# writer.writerows(rows)
# f.close()
# print('Saved to {}.csv'.format(csv_name))